In [254]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from numpy import asarray

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
import catboost as cb
from xgboost import XGBRegressor
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings('ignore')

In [61]:
crime = pd.read_csv("data/crimedata_cleaned.csv")

In [252]:
crime.head()

,incident_number,offense_code,crime_type,occurred_date_time,occurred_date,address,zip_code,location_type,x_coordinate,y_coordinate,longitude,latitude,council_district,month,year,week,dayofmonth,monthnum
0,20228022672,601,Burglary Of Vehicle,2022-12-01 11:30:00,2022-12-01,1921 Willow Creek Dr,78741,Residence / Home,3119469,10058001,-97.727425,30.232680,3,December,2022,Thursday,1,12
1,20225033150,504,Burglary Of Shed/Detached Garage/Storage Unit,2022-11-07 20:00:00,2022-11-07,4113 Avenue F,78751,Parking /Drop Lot/ Garage,3118205,10084145,-97.729489,30.304627,9,November,2022,Monday,7,11
2,20228022553,619,Theft Of Metal,2022-12-01 23:27:00,2022-12-01,300 W 6Th St,78701,Commercial / Office Building,3113404,10071112,-97.745659,30.269110,9,December,2022,Thursday,1,12
3,20228022692,2405,Doc Unreasonable Noise,2022-12-03 05:00:00,2022-12-03,11704 N Lamar Blvd,78758,Residence / Home,3131678,10114301,-97.684531,30.386647,7,December,2022,Saturday,3,12
4,20225031961,600,Theft,2022-10-29 23:00:00,2022-10-29,708 E 6Th St,78701,Residence / Home,3116608,10070123,-97.735585,30.266186,9,October,2022,Saturday,29,10


In [19]:
bnb = pd.read_csv("data/airbnb_cleaned.csv")

In [20]:
bnb.head()

,id,host_id,zip_code,neighborhood,property_type,room_type,accommodates,bedrooms,beds,minimum_nights,rating,price,longitude,latitude,availability_365
0,32666944,23,78744,McKinney,Private room in home,Private room,2,1.0,1.0,2,4.77,98,-97.75119,30.17937,241
1,23629432,23,78744,McKinney,Private room in home,Private room,2,1.0,1.0,1,4.46,80,-97.75080,30.17879,265
2,39056933,23,78744,McKinney,Tent,Entire home/apt,2,1.0,1.0,1,4.94,40,-97.75093,30.18005,148
3,42625339,796,78725,Austins' Colony,Entire cottage,Entire home/apt,4,2.0,4.0,1,4.93,138,-97.57637,30.24147,125
4,17074156,2466,78702,East Downtown,Entire guesthouse,Entire home/apt,3,1.0,1.0,30,4.68,135,-97.71234,30.27651,80


In [42]:
x = crime.groupby('crime_type').agg(Total=('crime_type','size')).sort_values(by='Total',ascending=False).reset_index()
crime_list_6600 = x[x.Total>=6600]
crime_list = crime_list_6600.crime_type.to_list()
crime_list

['Burglary Of Vehicle',
 'Theft',
 'Family Disturbance',
 'Criminal Mischief',
 'Assault With Injury',
 'Burglary Of Residence',
 'DWI',
 'Harassment',
 'Disturbance - Other',
 'Criminal Trespass',
 'Auto Theft',
 'Theft By Shoplifting',
 'Assault By Contact',
 'Custody Arrest Traffic Warr',
 'Narcotic Possession',
 'Warrant Arrest Non Traffic',
 'Burglary Non Residence',
 'Possession Of Marijuana',
 'Possession of Drug',
 'Aggravated Assault',
 'Debit/Credit Card Abuse',
 'Identity Theft',
 'Assault By Threat',
 'Theft Of Bicycle',
 'Violation City Ordinance',
 'Terroristic Threat',
 'Aggravated Robbery',
 'Fraud - Other',
 'Forgery And Passing',
 'Theft Of Service',
 'Dating Disturbance',
 'Theft From Auto',
 'Graffiti',
 'Voco - Alcohol  Consumption',
 'Child Custody Interfere',
 'Theft Of License Plate',
 'Failure To Identify',
 'Theft From Person',
 'Protective Order',
 'Viol State Law - Other',
 'Crash/Fail Stop And Render Aid',
 'Damage City Prop']

In [57]:
crime_df = crime[crime.crime_type.isin(crime_list)]
crime_df.shape

(2044001, 17)

In [48]:
xx = bnb.zip_code.value_counts().sort_index()
bnb_zip = xx.index.to_list()

In [59]:
crime_df = crime_df[crime_df.zip_code.isin(bnb_zip)]

In [60]:
crime_df.shape

(2012229, 17)

In [ ]:
filter = (bnb.property_type=='Room in hotel') | (bnb.room_type=='Hotel room') | (bnb.room_type=='Room in boutique hotel')
bnb = bnb[~filter]

Drop the row with the max price.

In [58]:
bnb.drop(bnb[bnb["price"]==bnb.price.max()].index,inplace=True)

In [253]:
bnb.describe()

,id,host_id,zip_code,accommodates,bedrooms,beds,minimum_nights,rating,price,longitude,latitude,availability_365
count,1.394800e+04,1.394800e+04,13948.000000,13948.000000,13948.000000,13948.000000,13948.000000,13948.000000,13948.000000,13948.000000,13948.000000,13948.000000
mean,2.655925e+17,1.334309e+08,78724.774448,4.818755,2.014482,2.621379,7.363421,4.815972,299.333883,-97.750495,30.281107,150.326355
std,3.485294e+17,1.492520e+08,20.641245,3.162548,1.284203,2.519836,24.920513,0.471687,465.976538,0.063032,0.064849,136.759281
min,5.456000e+03,2.300000e+01,78701.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,-98.056630,30.078870,0.000000
25%,2.317079e+07,1.578249e+07,78704.000000,2.000000,1.000000,1.000000,1.000000,4.830000,107.000000,-97.770533,30.241560,0.000000
50%,4.944158e+07,6.292611e+07,78723.000000,4.000000,2.000000,2.000000,2.000000,4.930000,180.000000,-97.740745,30.268601,123.000000
75%,6.711011e+17,2.208430e+08,78745.000000,6.000000,3.000000,3.000000,3.000000,5.000000,320.000000,-97.716920,30.309522,298.250000
max,8.473047e+17,5.051848e+08,78759.000000,16.000000,23.000000,132.000000,1100.000000,5.000000,10450.000000,-97.560620,30.512250,365.000000


Split the data set into training and testing.

In [ ]:
X=bnb.drop(['id','host_id','price'],axis=1)
y=bnb.price

**Let's look at the few algorithms on how they will predict the price without any hyperparameter tuning.**

In [256]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## CatBoost algorithm

In [257]:
cat_features = ['neighborhood','property_type','room_type']
model = cb.CatBoostRegressor(iterations=100, learning_rate=0.1, random_seed=42,cat_features=cat_features)

# Fit the model on the training data
model.fit(X_train, y_train, verbose=False)

# Make predictions on the test data
y_pred = model.predict(X_test)

In [258]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred, squared=False))
rmse

19.38665700010689

## XGBoost

Encode the categorical data to numerical.

In [261]:
m1 = LabelEncoder()
mapping = ['neighborhood','property_type','room_type']
X[mapping] = X[mapping].apply(m1.fit_transform)

In [262]:
model = XGBRegressor()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [263]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred, squared=False))
rmse

19.71124087049619

## LightGBM

In [265]:
# Initialize the LGBMRegressor model
model = lgb.LGBMRegressor(
    objective='regression',
    metric='rmse',
    num_leaves=31,
    learning_rate=0.1,
    feature_fraction=0.9
)

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)

[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9


In [266]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred, squared=False))
rms

19.162632211135076

## RandomForestRegressor

In [267]:
# Initialize the RandomForestRegressor model
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)

In [268]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred, squared=False))
rmse

19.991116518060462

## Linear Regression

In [269]:
# Initialize the LinearRegression model
model = LinearRegression()

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)

In [270]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred, squared=False))
rmse

20.499194573752515

LightGBM has low root mean squared error compared to other regression techniques.

We'll merge the crime data set with airbnb's data set and do the modeling later.